## Import

In [284]:
import pandas as pd
import talib
import backtrader as bt
from dataclasses import dataclass, asdict

## extract

In [285]:
daily: pd.DataFrame = pd.read_excel('gbp_usd_1D.xlsx')
h1: pd.DataFrame = pd.read_excel('gbp_usd_1h.xlsx')



## transform

### definitions

In [286]:
@dataclass
class TradeSignal:
    order_type: str
    entry_price: float
    sl: float
    tp0: float
    tp1: float
    tp2: float


def set_candle_type(r):
    ratio = r['true_range'] / r['atr']

    if ratio < 0.8:
        return 'spinning'
    if 1.2 > ratio >= 0.8:
        return 'standard'
    if 2.4 > ratio >= 1.2:
        return 'long'
    if ratio >= 2.4:
        return 'spike'


def get_candle_type_id(candle_type):
    if candle_type == 'spinning':
        return 0
    if candle_type == 'standard':
        return 1
    if candle_type == 'long':
        return 2
    if candle_type == 'spike':
        return 3


def candle_color(r):
    if r['open'] > r['close']:
        return 'r'
    if r['open'] < r['close']:
        return 'g'

    return 'y'


def set_ma_status(row):
    if row['close'] > row['ma']:
        return 'upper'
    if row['close'] < row['ma']:
        return 'under'
    return 'eq'




### indicators

In [287]:
h1['atr'] = talib.ATR(
    close=h1['close'],
    high=h1['high'],
    low=h1['low'],
    timeperiod=24
)
h1['rsi'] = talib.RSI(
    real=h1['close'],
    timeperiod=9
)
h1['ma'] = talib.MA(
    real=h1['close'],
    timeperiod=24
)
h1['true_range'] = talib.TRANGE(
    close=h1['close'],
    high=h1['high'],
    low=h1['low'],
)


### info

In [288]:
h1['range'] = h1['high'] - h1['low']
h1['candle_color'] = h1.apply(candle_color, axis=1)
h1['candle_type'] = h1.apply(set_candle_type, axis=1)
h1['candle_type_id'] = h1['candle_type'].apply(get_candle_type_id)
h1['ma_status'] = h1.apply(set_ma_status, axis=1)
h1['datetime'] = pd.to_datetime(h1['datetime'])
h1['signal'] = pd.NA


### drop Nones

In [289]:
h1 = h1.iloc[24:]
h1 = h1.drop(columns=['Unnamed: 0'])
h1 = h1.reset_index(drop=True)


### add signals

In [290]:

ma_status = h1.at[0, 'ma_status']
for i, element in h1.iterrows():

    if ma_status == element['ma_status']:
        continue

    ma_status = element['ma_status']

    h1.loc[i, 'signal'] = 'buy' if ma_status == 'upper' else 'sell'
h1['signal'] = h1['signal'].fillna(0)
h1['signal_id'] = h1['signal'].apply(lambda s: -1 if s == 'sell' else 1)

In [ ]:
h1['id'] = h1.index

In [340]:
class CustomPandasData(bt.feeds.PandasData):
    lines = ('atr', 'ma', 'signal_id', 'candle_type_id', 'rsi','id')  # add new lines
    params = (
        ('signal_id', -1),
        ('atr', -1),
        ('ma', -1),
        ('candle_type_id', -1),
        ('rsi', -1),
        ('id', -1),
    )


In [361]:
class StrategyA(bt.Strategy):
    
    def __init__(self):
        self.trades = []
        self.orders = []
        self._open_trades = {}
        
        self.call_counter = 0

    def calculate_volume(
            self,
            entry_price: float,
            stop_price: float,
            balance: int = 10000,
            risk: int = 1,
    ): 
        print("call_size")
        self.call_counter += 1
        risk_amount = balance * (risk / 100)
        per_unit_risk = abs(entry_price - stop_price)
        units = risk_amount / per_unit_risk
        return int(units)

    def next(self):
        cal_ma_status = lambda close, ma: 'upper' if close > ma else 'under'

        current_ma_status = cal_ma_status(self.data.close[0], self.data.ma[0])
        previous_ma_status = cal_ma_status(self.data.close[-1], self.data.ma[-1])

        if current_ma_status == previous_ma_status:
            return
 
        if current_ma_status == "upper":
            price = self.data.close[0]
            entry_price = price
            stop_loss = entry_price - self.data.atr[0] * 2.4
            take_profit = entry_price + self.data.atr[0] * 4.8
            size = self.calculate_volume(
                    entry_price,
                    stop_loss,
                    risk=2
                )
            self.buy_bracket(
                price=entry_price,
                stopprice=stop_loss,
                limitprice=take_profit,
                size=size,
                tradeid = self.data.id[0]
            )
        
        # if current_ma_status == "under":
        #     price = self.data.close[0]
        #     entry_price = price
        #     stop_loss = entry_price + self.data.atr[0] * 2.4
        #     take_profit = entry_price - self.data.atr[0] * 4.8
        #     size = self.calculate_volume(
        #             entry_price,
        #             stop_loss,
        #             risk=2
        #         )
        #     self.sell_bracket(
        #         price=entry_price,
        #         stopprice=stop_loss,
        #         limitprice=take_profit,
        #         size=size
        #     )

    def notify_trade(self, trade : bt.trade.Trade):
        self.trades.append(trade)
        

    def notify_order(self, order):
        self.orders.append(order)



In [362]:
data = CustomPandasData(dataname=h1.set_index("datetime"))

cerebro = bt.Cerebro()
cerebro.adddata(data)
cerebro.addstrategy(StrategyA)
cerebro.addanalyzer(bt.analyzers.TradeAnalyzer, _name='trade')
cerebro.broker.setcash(10000)
cerebro.broker.set_coo(True)
cerebro.broker.setcommission(commission=0.0, leverage=500)
result = cerebro.run()

call_size
call_size
call_size
call_size
call_size
call_size
call_size
call_size
call_size
call_size
call_size
call_size
call_size
call_size
call_size
call_size
call_size
call_size
call_size
call_size
call_size
call_size
call_size
call_size
call_size
call_size
call_size
call_size
call_size
call_size
call_size
call_size
call_size
call_size
call_size
call_size
call_size
call_size
call_size
call_size
call_size
call_size
call_size
call_size
call_size
call_size
call_size
call_size
call_size
call_size
call_size
call_size
call_size
call_size
call_size
call_size
call_size
call_size
call_size
call_size
call_size
call_size
call_size
call_size
call_size
call_size
call_size
call_size
call_size
call_size
call_size
call_size
call_size
call_size
call_size
call_size
call_size
call_size
call_size
call_size
call_size
call_size
call_size
call_size
call_size
call_size
call_size
call_size
call_size
call_size
call_size
call_size
call_size
call_size
call_size
call_size
call_size
call_size
call_size
call_size


In [363]:
cerebro.broker.getvalue()

34966.027487161795

In [364]:
strategy_a_result = result[0]

In [366]:
len(strategy_a_result.trades)

3833

In [381]:
trades = [{'pnl' : trade.pnl , 'tradeid' : trade.tradeid} for trade in strategy_a_result.trades if trade.pnl != 0.0]

        

In [384]:
trades_df = pd.DataFrame(trades)

In [386]:
trading_buy_data = pd.merge(h1, trades_df, left_on='id' , right_on='tradeid', how='inner')

In [388]:
trading_buy_data['is_target'] = trading_buy_data['pnl'].apply(lambda pnl : 1 if pnl > 0 else 0)

In [390]:
trading_buy_data = trading_buy_data[['datetime', 'rsi', 'candle_type_id', 'is_target']]

In [408]:
len(trading_buy_data[trading_buy_data['is_target'] == 0])

1235

In [423]:
len(trading_buy_data[(trading_buy_data['is_target'] == 1) & (trading_buy_data['candle_type_id'] == 0.0)]) / len(trading_buy_data[(trading_buy_data['is_target'] == 0) & (trading_buy_data['candle_type_id'] == 0.0)])

0.6013071895424836

In [424]:
len(trading_buy_data[(trading_buy_data['is_target'] == 1) & (trading_buy_data['candle_type_id'] == 1.0)]) / len(trading_buy_data[(trading_buy_data['is_target'] == 0) & (trading_buy_data['candle_type_id'] == 1.0)])

0.6666666666666666

In [425]:
len(trading_buy_data[(trading_buy_data['is_target'] == 1) & (trading_buy_data['candle_type_id'] == 2.0)]) / len(trading_buy_data[(trading_buy_data['is_target'] == 0) & (trading_buy_data['candle_type_id'] == 2.0)])

0.47752808988764045

In [426]:
len(trading_buy_data[(trading_buy_data['is_target'] == 1) & (trading_buy_data['candle_type_id'] == 3.0)]) / len(trading_buy_data[(trading_buy_data['is_target'] == 0) & (trading_buy_data['candle_type_id'] == 3.0)])

0.4423076923076923

In [428]:
trading_buy_data['hour'] = trading_buy_data['datetime'].apply(lambda d : int(str(d).split()[1].split(':')[0]))

C:\Users\Soheil\AppData\Local\Temp\ipykernel_11008\1959529976.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  trading_buy_data['hour'] = trading_buy_data['datetime'].apply(lambda d : int(str(d).split()[1].split(':')[0]))


In [431]:
trading_buy_data['in_session'] = trading_buy_data['hour'].apply(lambda h : 1 if h in range(4,16) else 0)

C:\Users\Soheil\AppData\Local\Temp\ipykernel_11008\4082188684.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  trading_buy_data['in_session'] = trading_buy_data['hour'].apply(lambda h : 1 if h in range(4,16) else 0)


In [442]:
len(trading_buy_data[(trading_buy_data['is_target'] == 0) & (trading_buy_data['in_session'] == 0)])

475

In [445]:
def get_rsi_range(rsi):
    if 30 <= rsi <= 70:
        return 0
    if rsi > 70:
        return 1
    if rsi < 30:
        return 2

In [446]:
trading_buy_data['rasi_range'] = trading_buy_data['rsi'].apply(get_rsi_range)

C:\Users\Soheil\AppData\Local\Temp\ipykernel_11008\939026108.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  trading_buy_data['rasi_range'] = trading_buy_data['rsi'].apply(get_rsi_range)


In [454]:
len(trading_buy_data[(trading_buy_data['is_target'] == 1) & (trading_buy_data['rasi_range'] == 2)])

0

In [455]:
trading_buy_data['rasi_range'].value_counts()

rasi_range
0    1877
1      37
Name: count, dtype: int64

In [464]:
len(trading_buy_data[(trading_buy_data['is_target'] == 1) & (trading_buy_data['candle_type_id'] == 0.0) & (trading_buy_data['in_session'] == 0)])

105

In [466]:
trading_buy_data

,datetime,rsi,candle_type_id,is_target,hour,in_session,rasi_range
0,2020-01-06 08:00:00+00:00,48.132626,0.0,1,8,1,0
1,2020-01-06 10:00:00+00:00,63.201091,2.0,1,10,1,0
2,2020-01-07 10:00:00+00:00,66.691108,3.0,0,10,1,0
3,2020-01-08 08:00:00+00:00,52.909515,1.0,0,8,1,0
4,2020-01-08 16:00:00+00:00,50.450487,1.0,0,16,0,0
...,...,...,...,...,...,...,...
1909,2025-09-23 14:00:00+00:00,54.703679,1.0,0,14,1,0
1910,2025-09-25 06:00:00+00:00,54.076361,0.0,0,6,1,0
1911,2025-09-25 09:00:00+00:00,56.481317,2.0,0,9,1,0
1912,2025-09-26 11:00:00+00:00,54.113173,2.0,1,11,1,0
